In [1]:
%matplotlib inline

In [2]:
import requests
import speech_recognition as sr
from os import path
from pprint import pprint
from IPython.display import HTML

In [13]:
sentiment_subscription_key = "e552bd5890934ed595dbf8d7fd05816f"
speech_to_text_key = "2ed41467970e4791bec9a48388bb416e"
search_subscription_key = "9db3ddee989743f48b738680d30c5bab"
text_analytics_base_url = "https://westus.api.cognitive.microsoft.com/text/analytics/v2.0/"
language_api_url = text_analytics_base_url + "languages"
sentiment_api_url = text_analytics_base_url + "sentiment"
key_phrase_api_url = text_analytics_base_url + "keyPhrases"
search_url = "https://api.cognitive.microsoft.com/bing/v7.0/search/"

In [4]:
def extract_text_from_audio(audio_path):
    audio_file = path.join(path.dirname(path.realpath('__file__')), audio_path)
    r = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = r.record(source)
    try:
        raw_text = r.recognize_bing(audio, key=speech_to_text_key)
        document = {'documents' : [
          {'id': '1', 'language': 'en', 'text': raw_text}
        ]}
        return(document)
    except sr.UnknownValueError:
        return("Microsoft Bing Voice Recognition could not understand audio")
    except sr.RequestError as e:
        return("Could not request results from Microsoft Bing Voice Recognition service; {0}".format(e))

In [5]:
def get_sentiment(document):
    headers   = {"Ocp-Apim-Subscription-Key": sentiment_subscription_key}
    response  = requests.post(sentiment_api_url, headers=headers, json=document)
    sentiments = response.json()
    return(sentiments['documents'][0]['score'])

In [6]:
def get_keywords(document):
    headers   = {"Ocp-Apim-Subscription-Key": sentiment_subscription_key}
    response  = requests.post(key_phrase_api_url, headers=headers, json=document)
    key_phrases = response.json()
    return(key_phrases['documents'][0]['keyPhrases'])

In [7]:
document = extract_text_from_audio(audio_path='sample.wav')
print(document)

{'documents': [{'id': '1', 'language': 'en', 'text': 'Every word and phrase he speaks is true he put his last cartridge into the gun and fired they took their kids from the public school drive the screws straight into the would keep the hatch tight in the watch constant several that wine with a quick snip of the knife paper will dry out when wet slide the catch back and open the desk help the week to preserve their strength they selling smile get you friends.'}]}


In [8]:
sentiment = get_sentiment(document)
keywords = get_keywords(document)
print(sentiment)
print(keywords)

0.8571277856826782
['watch constant', 'knife paper', 'quick snip', 'wine', 'wet slide', 'strength', 'kids', 'public school', 'catch', 'hatch tight', 'word', 'phrase', 'smile', 'friends', 'cartridge', 'week']


In [9]:
def get_search_results(search_term):
    headers = {"Ocp-Apim-Subscription-Key" : search_subscription_key}
    params  = {"q": search_term, "textDecorations":True, "textFormat":"HTML"}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    rows = "\n".join(["""<tr>
                       <td><a href=\"{0}\">{1}</a></td>
                       <td>{2}</td>
                     </tr>""".format(v["url"],v["name"],v["snippet"]) for v in search_results["webPages"]["value"]])
    return HTML("<table>{0}</table>".format(rows))

In [14]:
get_search_results(keywords[0])

"Amazon.com: Frederique Constant: Clothing, Shoes & Jewelry","Frederique Constant. FC245M4S6 Men's Slim Line Analog Display Swiss Quartz Watch, Black $ 316 79 Prime. 4.8 out of 5 stars 19. ... shoes, jewelry, watches, and more."
Frédérique Constant Watches - Authorized Retailer - Tourneau,"Shop authorized Frédérique Constant watch retailer - w/ manufacturer warranty and 5-yr Tourneau warranty. Classics, Manufacture, Slimline & more."
Frederique Constant Watches - Jomashop,Free shipping on all Frederique Constant Watches over $100. Jomashop.com features a huge selection of authentic Frederique Constant Watches at low prices.... - On Sale:
Frederique Constant Watches - Walmart.com,"Shop for Frederique Constant Watches in Jewelry. Buy products such as Frederique Constant Classics Men's Watch, FC-306MC4S35 at Walmart and save."
Frederique Constant watches - Ashford.com,Ashford Luxury Watches. Watches for sale at Ashford.com are shipped with a reputation for integrity and superiority in the world of luxury watches.
Frederique Constant Watches For Less | Overstock.com,"Frederique Constant Watches : Find top watch brands like Michael Kors, Rolex, Gucci, Citizen, Fossil, Coach, Omega, and more! Overstock.com - Get 5% in rewards with ..."
Frederique Constant - Gnomon Watches,"At Gnomon Watches you'll find a huge range of German & Swiss watches at leading price. Includes Watches : Archimede, Aristo, Aviation, Damasko, Dievas, Ennebi, Laco ..."
frederique constant watch | eBay,Find great deals on eBay for frederique constant watch and raymond weil watch. Shop with confidence.
Frederique Constant | Bloomingdale's,FREDERIQUE CONSTANT WATCHES. Wear Frederique Constant watches as elegant adornments and statement pieces. In a world that expects more from a watch than just telling ...
Frédérique Constant - Wikipedia,"Frédérique Constant SA is a manufacture of luxury wrist watches based in Plan-les-Ouates, Geneva. It was acquired in 2017 by Citizen Holdings of Tokyo, Japan. The ..."
